# TP3 - NLP

#### CoNLL-U parser

In [1]:
from io import open
def parser(file):
    # file should be .conllu
    data_file = open(file, "r", encoding="utf-8")
    res_file = ""
    for j in data_file:
        res_file+=j
    res = res_file.split('\n')
    for i in range(len(res)):
        res[i] = res[i].split()
    res_list = []
    i_list = 0
    for line in res:
        if not line:
            i_list+=1
            continue
        if line[0] == '#':
            continue
        if line[0] == '1':
            res_list.append([(line[1],line[3])])
            continue
        if 48 < ord(line[0][0]) < 58 and '-' not in line[0]:
            # here this condition is used to fix the issue when we have a number with space, ex: 48 230
            if 47 < ord(line[1][0]) < 58 and 47 < ord(line[2][0]) < 58 and 47 < ord(line[3][0]) < 58:
                res_list[i_list].append((line[1],"NUM"))
                continue
            res_list[i_list].append((line[1],line[3]))
    return res_list

## Lab3

In [2]:
from sklearn.linear_model import Perceptron
import numpy as np
from sklearn.decomposition import PCA

In [3]:
res_french_train = parser("UD_French-GSD/fr_gsd-ud-train.conllu")
res_french_test = parser("UD_French-GSD/fr_gsd-ud-test.conllu")

In [4]:
X_train_french = [[word] for wt in res_french_train for word,tag in wt]
Y_train_french = [[tag] for wt in res_french_train for word,tag in wt]

X_test_french = [[word] for wt in res_french_test for word,tag in wt]
Y_test_french = [[tag] for wt in res_french_test for word,tag in wt]

In [5]:
X_train_fr = np.array(X_train_french)
Y_train_fr= np.array(Y_train_french)

X_test_fr = np.array(X_test_french)
Y_test_fr= np.array(Y_test_french)

#### Here we start with the length of the word as basic feature

In [6]:
len_np = np.vectorize(len)
X_train = len_np(X_train_fr)

X_test = len_np(X_test_fr)

In [7]:
pp = Perceptron().fit(X_train,Y_train_fr)
res_pred = pp.predict(X_test)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [8]:
nb = 0
for i in range(len(res_pred)):
    if Y_test_fr[i] == res_pred[i]:
        nb+=1
res_length = nb/i
res_length

0.02486518873576992

--------------

#### Now we use feature based on gramatical rule of the word

In [9]:
def simple_feature_french_language(word):
    tag = "NOUN"
    if word.endswith("s"):
        tag = "NOUN"
    elif word.endswith("ment"):
        tag = "ADV"
    elif word.endswith("er"):
        tag = "VERB"
    elif word.endswith("e"):
        tag = "NOUN"
    if word.startswith("dé"):
        tag = "VERB"
    elif word.startswith("pré"):
        tag = "NOUN"
    elif word.startswith("sous"):
        tag = "ADP"
    if word.endswith("ion"):
        tag = "NOUN"
    elif word.endswith("ier"):
        tag = "NOUN"
    elif word.endswith("ité"):
        tag = "NOUN"
    if word == "est":
        tag = "VERB"
    elif word == "et":
        tag = "CCONJ"
    elif word == "de":
        tag = "ADP"
    elif 47 < ord(word[0]) < 58:
        tag = "NUM"
    return tag

def length_word_feature(word):
    return len(word)

In [10]:
Y_train_fr.shape
res_y = {}
num_y = {}
num = 0
for i in range(len(Y_train_fr)):
    if Y_train_fr[i][0] not in res_y:
        res_y[Y_train_fr[i][0]] = 1
        num_y[Y_train_fr[i][0]] = num
        num+=1
    else:
        res_y[Y_train_fr[i][0]] += 1

In [11]:
feat_wd = np.vectorize(simple_feature_french_language)
X_train = feat_wd(X_train_fr)
X_test = feat_wd(X_test_fr)

In [12]:
y_c = np.arange(16)
y_hot_coding = np.zeros((y_c.size, y_c.max()+1))
y_hot_coding[np.arange(y_c.size), y_c] = 1

In [13]:
def to_vect(X_TR):
    res = []
    for x in X_TR:
        nb = num_y[x[0]]
        res.append(y_hot_coding[nb])
    return np.array(res)

In [14]:
X_train_nb = to_vect(X_train)
X_test_nb = to_vect(X_test)

In [15]:
pp = Perceptron().fit(X_train_nb,Y_train_fr)
res_pred = pp.predict(X_test_nb)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [16]:
nb = 0
for i in range(len(res_pred)):
    if Y_test_fr[i][0] == res_pred[i]:
        nb+=1
res_gram = nb/i
res_gram

0.15688036748552028

----------------

#### Here we combine the 2 features

In [17]:
len_np = np.vectorize(len)
X_train_f = len_np(X_train_fr)

X_test_f = len_np(X_test_fr)

In [18]:
X_train_concat = np.concatenate((X_train_nb, X_train_f), axis=1)
X_test_concat = np.concatenate((X_test_nb, X_test_f), axis=1)

In [19]:
pp = Perceptron().fit(X_train_concat,Y_train_fr)
res_pred = pp.predict(X_test_concat)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [20]:
nb = 0
for i in range(len(res_pred)):
    if Y_test_fr[i][0] == res_pred[i]:
        nb+=1
res_concat = nb/i
res_concat

0.18094667465548234

-------------------------

#### Here we use the pca matrix to reduce the dimension of the features

In [21]:
pca = PCA(n_components=4)
pca.fit(X_train_concat)

PCA(n_components=4)

In [22]:
X_train_pca = pca.fit_transform(X_train_concat)

In [23]:
X_test_pca = pca.fit_transform(X_test_concat)

In [24]:
pp = Perceptron().fit(X_train_pca,Y_train_fr)
res_pred = pp.predict(X_test_pca)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [25]:
nb = 0
for i in range(len(res_pred)):
    if Y_test_fr[i][0] == res_pred[i]:
        nb+=1
res_pca = nb/i
res_pca

0.33992410625124825

-----------------

#### Here we add to the pca matrix a feature of a rule using the left and the right neighbors

In [26]:
res_neigb = []
for sentence in res_french_train:
    for i, (word,tag) in enumerate(sentence):
        if i == 0:
            res_neigb.append([(word,tag),(word,tag),sentence[i+1]])
        elif i == len(sentence)-1:
            res_neigb.append([sentence[i-1],(word,tag),(word,tag)])
        else:
            res_neigb.append([sentence[i-1],(word,tag),sentence[i+1]])

In [27]:
test_neigb = []
for sentence in res_french_test:
    for i, (word,tag) in enumerate(sentence):
        if i == 0:
            test_neigb.append([(word,tag),(word,tag),sentence[i+1]])
        elif i == len(sentence)-1:
            test_neigb.append([sentence[i-1],(word,tag),(word,tag)])
        else:
            test_neigb.append([sentence[i-1],(word,tag),sentence[i+1]])

In [28]:
def neighbor_feature(word_list):
    tag = "NOUN"
    word = word_list[1][0]
    left_word = word_list[0][0]
    right_word = word_list[2][0]
    right_tag = word_list[2][1]
    left_tag = word_list[0][1]
    
    if left_word == word and right_tag in ['ADJ', 'NOUN']:
        tag = 'DET'
    if left_tag == "DET" and right_tag == "ADJ":
        tag = "NOUN"
    elif left_tag == "DET" and right_tag == "VERB":
        tag = "NOUN"
    elif left_tag == "DET" and right_tag == "NOUN":
        tag = "ADJ"
    elif left_tag == "DET":
        tag = "NOUN"
    elif word.endswith("ment") and right_word in ["de", "à"]:
        tag = "ADV"
    elif right_word == "NOUN":
        tag = "DET"
    elif 47 < ord(left_word[0]) < 58:
        tag = "NOUN"
    return tag

In [29]:
x_neib = []
for seq in res_neigb:
    x_neib.append(neighbor_feature(seq))

In [30]:
x_neib_t = []
for seq in test_neigb:
    x_neib_t.append(neighbor_feature(seq))

In [31]:
X_train_tri = np.array(x_neib)

In [32]:
X_test_tri = np.array(x_neib_t)

In [33]:
X_train_nb_tri = []
for j in X_train_tri:
    nb = num_y[j]
    X_train_nb_tri.append(y_hot_coding[nb])
X_train_nb_tri = np.array(X_train_nb_tri)

In [34]:
X_test_nb_tri = []
for j in X_test_tri:
    nb = num_y[j]
    X_test_nb_tri.append(y_hot_coding[nb])
X_test_nb_tri = np.array(X_test_nb_tri)

In [35]:
pca = PCA(n_components=2)
pca.fit(X_train_nb_tri)

PCA(n_components=2)

In [36]:
X_train_pca_nb = pca.fit_transform(X_train_nb_tri)

In [37]:
X_test_pca_nb = pca.fit_transform(X_test_nb_tri)

In [38]:
X_train_concat_nb = np.concatenate((X_train_pca, X_train_pca_nb), axis=1)

In [39]:
X_test_concat_nb = np.concatenate((X_test_pca, X_test_pca_nb), axis=1)

In [40]:
pp = Perceptron().fit(X_train_concat_nb,Y_train_fr)
res_pred = pp.predict(X_test_concat_nb)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [41]:
nb = 0
for i in range(len(res_pred)):
    if Y_test_fr[i][0] == res_pred[i]:
        nb+=1
res_pca_rule_neighbor = nb/i
res_pca_rule_neighbor

0.20311563810665068

-----------------------

#### Here we add to the pca matrix, the feature of the left and right neighbors

In [42]:
neig_left = []
for h in range(len(res_neigb)):
    neig_left.append(res_neigb[h][0][0])


neig_right = []
for h in range(len(res_neigb)):
    neig_right.append(res_neigb[h][2][0])


neig_left_test = []
for h in range(len(test_neigb)):
    neig_left_test.append(test_neigb[h][0][0])


neig_right_test = []
for h in range(len(test_neigb)):
    neig_right_test.append(test_neigb[h][2][0])

In [43]:
X_train_left = feat_wd(neig_left)
X_test_left = feat_wd(neig_left_test)

X_train_right = feat_wd(neig_right)
X_test_right = feat_wd(neig_right_test)

In [44]:
def nb_vt(xt):
    res = []
    for x in xt:
        nb = num_y[x]
        res.append(y_hot_coding[nb])
    return np.array(res)

In [45]:
X_train_right = nb_vt(X_train_right)
X_test_right = nb_vt(X_test_right)

X_train_left = nb_vt(X_train_left)
X_test_left = nb_vt(X_test_left)

In [46]:
pca = PCA(n_components=2)
pca.fit(X_train_right)

X_train_right = pca.fit_transform(X_train_right)
X_train_left = pca.fit_transform(X_train_left)

X_test_right = pca.fit_transform(X_test_right)
X_test_left = pca.fit_transform(X_test_left)

In [47]:
X_train_concat_rt = np.concatenate((X_train_pca, X_train_right), axis=1)
X_train_concat_rt = np.concatenate((X_train_concat_rt, X_train_left), axis=1)

X_test_concat_rt = np.concatenate((X_test_pca, X_test_right), axis=1)
X_test_concat_rt = np.concatenate((X_test_concat_rt, X_test_left), axis=1)

In [48]:
pp = Perceptron().fit(X_train_concat_rt,Y_train_fr)
res_pred = pp.predict(X_test_concat_rt)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [49]:
nb = 0
for i in range(len(res_pred)):
    if Y_test_fr[i][0] == res_pred[i]:
        nb+=1
res_fin_pca_left_right = nb/i
res_fin_pca_left_right

0.2353704813261434

--------

----------------

# Conclusion

Here, in this notebook, we only work on the French table, since we are late on the project and so we wanted to test the different possible combinations for the features, since to test the English one's, we have to add new grammar rules, but no time remain for us, and we are already late.
So we tried to write some grammar rules to create the features for the french table.
Here are the results:

In [50]:
print(f"Accuracy using only the length feature: {round(res_length,3)*100}%") 
print(f"Accuracy using only the grammatical rules feature: {round(res_gram,3)*100}%") 
print(f"Accuracy using the combination between the length and grammatical features: {round(res_concat,2)*100}%") 
print(f"Accuracy using pca on the concatenation: {round(res_pca,3)*100}%")
print(f"Accuracy using the combination between the pca and the grammatical rules on the neighbor: {round(res_pca_rule_neighbor,3)*100}%") 
print(f"Accuracy using the combination between the pca and the grammatical feature of the left and right neighbors: {round(res_fin_pca_left_right,3)*100}%") 

Accuracy using only the length feature: 2.5%
Accuracy using only the grammatical rules feature: 15.7%
Accuracy using the combination between the length and grammatical features: 18.0%
Accuracy using pca on the concatenation: 34.0%
Accuracy using the combination between the pca and the grammatical rules on the neighbor: 20.3%
Accuracy using the combination between the pca and the grammatical feature of the left and right neighbors: 23.5%


The best value obtained is that of the pca after concatenation of the two simple features, and we can see that by using
grammatical rules the result is better than a naive method like the length of the word.
But what is not expected is that the accuracy will decrease when adding grammatical rules using neighbors,
or when just adding the features of the neighbors.
However, in our case this is understandable since our grammatical function is not relevant enough.
If we have more time we'll concentrate more on the part of creating grammatical rules for both words and neighbors.